# Response Item Network Analysis

## Import

In [ ]:
# Import 
import pandas as pd


# Import custom libraries
import util
from util import UtilityFunctions as uf

from importlib import reload
reload(util)   

import responseItemNetwork as ResIN

# Monkey patch the method from the utility class to the pandas DataFrame
pd.DataFrame.filter_features = uf.filter_features
pd.DataFrame.filter_numerical_values = uf.filter_numerical_values
pd.DataFrame.filter_negative_values = uf.filter_negative_values
pd.DataFrame.filter_columns_with_less_unique_values_than_threshold = uf.filter_columns_with_less_unique_values_than_threshold
pd.DataFrame.drop_columns = uf.drop_columns

## Load Data 

In [ ]:
path = './../data/processed/'
file_to_load = "2018_U"
processed_files = uf.get_csv_files_from_folder(path)
for f in processed_files:
        if f.startswith(file_to_load):
            print(f)
            df = pd.read_csv(f)

## Select Features

In [ ]:
list_of_questions = ['E035','E036','E036', 'E035','E033']
df_filtered = df.filter_features(list_of_questions)

## Process Features Into Binary variables 

In [ ]:
for 